In [22]:
#%%
from model import Generator, Discriminator, DomainClassifier
from utils import audio_process
import IPython.display as ipd
import tensorflow as tf
import numpy as np
import os

In [23]:
#%%
'''
Usage:
  Interactive web page via jupyter
'''
ckpt_path = './ckpt_gam_mel_150k'

G = Generator()
dummy_wav = np.zeros((1, 36, 512, 1), dtype=np.float32)
dummy_speaker_onehot = np.zeros((1, 2), dtype=np.float32)
G(dummy_wav, dummy_speaker_onehot, True)
restored_ckpt = tf.train.Checkpoint(G=G)
latest_ckpt = tf.train.latest_checkpoint(ckpt_path)
restored_ckpt.restore(latest_ckpt).assert_existing_objects_matched()

In [25]:
#%%
result_dir = 'result'
input_wav = '../dataset/speakers/p2/p2_260.wav'
target_speaker = 0
speaker_counts = 2

_, mel = audio_process.wav2spectrogram(input_wav)
mel = mel.astype(np.float32)
mel = mel[np.newaxis, :, :, np.newaxis]
speaker =np.eye(speaker_counts, dtype=np.float32)[target_speaker]
speaker = speaker[np.newaxis, :]
print(speaker)
print(np.mean(mel))
converted = G(mel, speaker, True)
converted_np = converted.numpy()[0,:,:,0]
print(np.mean(converted_np))
converted_wav = audio_process.melspectrogram2wav(converted_np)

filename = input_wav.split('/')[-1].split('.')[0]
filename = filename + '_to_speaker%d.wav' % (target_speaker)
filepath = os.path.join(ckpt_path, result_dir, filename)
os.makedirs(os.path.dirname(filepath), exist_ok=True)
abs_filepath = audio_process.write_wav(converted_wav, filepath)
ipd.Audio(abs_filepath)

[[1. 0.]]
0.011872848
0.0070098056
